In [81]:
import pandas as pd
import numpy as np
import random 
import os
def random_data(): 
    return None
def get_domains(data):
    return None

In [82]:
class Data:

	def generate_random_data(self, variable_data, N=10):
  
		data = {}
		data['id'] = range(0, N)
		
		for i, variable in enumerate(variable_data):
			name = variable['name']
			var_type = variable['type']
			step = float(variable['step']) if variable['step'] else 0
   
			if var_type == 'boolean':
				data[name] = [random.choice([False, True]) for _ in range(N)]
			elif var_type == 'int':
				min_value = int(variable['min'])
				max_value = int(variable['max'])
				data[name]  = [random.randint(int(min_value), int(max_value)) for _ in range(N)]
			elif var_type == 'float':
				min_value = float(variable['min'])
				max_value = float(variable['max'])
				data[name] = [round(random.uniform(float(min_value), float(max_value)), int(step)) for _ in range(N)]
	
		df = pd.DataFrame(data)
		return df

	def __init__(self, variable_data, N):
		self.data = pd.DataFrame(self.generate_random_data(variable_data, N))
		self.__domains = get_domains(self.data)

	def get_values(self, id1, id2=None, X=[]):
		if id2 is None:
			if X == [] or X == self.data.keys():
				return self.data[self.data.id == id1]  # retourner toutes les valeurs
			else:
				return self.data[self.data.id == id1][X].to_dict()  # retourner les valeurs selectionnées

		else:
			if X == [] or X == self.data.keys():
				return self.data[self.data.id == id1], self.data[self.data.id == id2]  # retourner toutes les valeurs
			else:
				return self.data[self.data.id == id1][X].to_dict(), self.data[self.data.id == id2][X].to_dict()

	def V(self):
		return self.__domains.keys()


In [85]:
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk
import numpy as np
import pandas as pd

WINDOW_WIDTH = 800
WINDOW_HEIGHT = 600

# Define global variables for entry, dropdown, and treeview
variable_entry = None
variable_type_var = None
min_entry = None
max_entry = None
step_entry = None
variables_tree = None
variable_counter = 0  # Counter for generating default variable names

# Define global labels for min, max, and step
min_label = None
max_label = None
step_label = None

def import_csv():
    file_path = filedialog.askopenfilename()
    if file_path:
        # Open GUI for selecting variables
        select_variables_window()

def generate_random():
    # Open GUI for selecting variables
    select_variables_window()

def add_variable():
    global variable_entry, variable_type_var, min_entry, max_entry, step_entry, variable_counter, variables_tree

    suggested_name = f"x{variable_counter + 1}"  # Suggest default variable name
    variable_name = variable_entry.get() or suggested_name
    variable_type = variable_type_var.get()
    toggle_min_max_step_visibility(variable_type)
    min_value = min_entry.get() if min_entry.get() else get_default_min_value(variable_type)
    max_value = max_entry.get() if max_entry.get() else get_default_max_value(variable_type)
    step_value = step_entry.get() if step_entry.get() else get_default_step_value(variable_type)

    if variable_name:
        variables_tree.insert("", "end", values=(variable_name, variable_type, min_value, max_value, step_value))

        # Increment counter for next variable name suggestion
        variable_counter += 1

        variable_entry.delete(0, tk.END)

def select_variables_window():
    global variable_entry, variable_type_var, min_entry, max_entry, step_entry, num_rows_entry, variables_tree, variables_window
    global min_label, max_label, step_label, next_button

    variables_window = tk.Toplevel(root)
    variables_window.title("Select Variables")
    variables_window.geometry(f"{WINDOW_WIDTH}x{WINDOW_HEIGHT}")

    # Entry for variable name
    variable_label = tk.Label(variables_window, text="Variable Name:")
    variable_label.grid(row=0, column=0, padx=5, pady=5)
    suggested_name = f"x{variable_counter + 1}"  # Suggest default variable name
    variable_entry = tk.Entry(variables_window)
    variable_entry.insert(0, suggested_name)  # Set suggested name as default value
    variable_entry.grid(row=0, column=1, padx=5, pady=5)

    # Dropdown for variable type
    type_label = tk.Label(variables_window, text="Variable Type:")
    type_label.grid(row=0, column=2, padx=5, pady=5)
    variable_type_var = tk.StringVar(variables_window)
    variable_type_var.set("int")  # Default type
    type_dropdown = tk.OptionMenu(variables_window, variable_type_var, "int", "float", "boolean", command=toggle_min_max_step_visibility)
    type_dropdown.grid(row=0, column=3, padx=5, pady=5)

    # Entry for min value (visible for int and float types)
    min_label = tk.Label(variables_window, text="Min:")
    min_label.grid(row=0, column=4, padx=5, pady=5)
    min_entry = tk.Entry(variables_window)
    min_entry.grid(row=0, column=5, padx=5, pady=5)

    # Entry for max value (visible for int and float types)
    max_label = tk.Label(variables_window, text="Max:")
    max_label.grid(row=0, column=6, padx=5, pady=5)
    max_entry = tk.Entry(variables_window)
    max_entry.grid(row=0, column=7, padx=5, pady=5)

    # Entry for step value (visible for float type)
    step_label = tk.Label(variables_window, text="Step:")
    step_label.grid(row=0, column=8, padx=5, pady=5)
    step_entry = tk.Entry(variables_window)
    step_entry.grid(row=0, column=9, padx=5, pady=5)

    # Button to add variable
    add_button = tk.Button(variables_window, text="Add", command=add_variable)
    add_button.grid(row=0, column=10, padx=5, pady=5)

    # Treeview to display added variables in a table
    variables_tree = ttk.Treeview(variables_window, columns=("Name", "Type", "Min", "Max", "Step"), show="headings")
    variables_tree.heading("Name", text="Name")
    variables_tree.heading("Type", text="Type")
    variables_tree.heading("Min", text="Min")
    variables_tree.heading("Max", text="Max")
    variables_tree.heading("Step", text="Step")
    variables_tree.grid(row=1, column=0, columnspan=11, padx=5, pady=5)
    
    num_rows_label = tk.Label(variables_window, text="N:")
    num_rows_label.grid(row=2, column=7, padx=5, pady=5)
    num_rows_entry = tk.Entry(variables_window)
    num_rows_entry.insert(0, "10")  # Default number of rows
    num_rows_entry.grid(row=2, column=8, padx=5, pady=5)
    
        # Button to add variable
    add_button2 = tk.Button(variables_window, text="Add", command=add_variable)
    add_button2.grid(row=2, column=9, padx=5, pady=5)

    # Button to go to next GUI
    next_button = tk.Button(variables_window, text="Generate", command=display_data)
    next_button.grid(row=2, column=10, columnspan=11, padx=5, pady=5)

    # Call toggle_min_max_step_visibility initially to set visibility based on default type
    toggle_min_max_step_visibility(variable_type_var.get())
    
def get_variable_data():
    variable_data = []
    for item in variables_tree.get_children():
        name = variables_tree.item(item, "values")[0]
        type = variables_tree.item(item, "values")[1]
        min_value = variables_tree.item(item, "values")[2]
        max_value = variables_tree.item(item, "values")[3]
        step_value = variables_tree.item(item, "values")[4]
        variable_data.append({"name": name, "type": type, "min": min_value, "max": max_value, "step": step_value})
    return [variable_data, int(num_rows_entry.get()) if num_rows_entry.get() else 10]

def display_data():
    variable_data, N = get_variable_data()
    
    # Create a Data object
    data_object = Data(variable_data)

    # Create a new window for the data table
    data_window = tk.Toplevel(root)
    data_window.title("Data Table")

    
    # Create a table/grid layout to display the data
    table = ttk.Treeview(data_window, columns=[variable['name'] for variable in variable_data])

    # Add columns for variable names
    for variable in variable_data:
        table.heading(variable['name'], text=variable['name'])

    # Add specified number of rows
    rows = int(num_rows_entry.get())
    for i in range(rows):
        table.insert("", "end", values=["" for _ in variable_data])

    table.grid(row=1, column=0, columnspan=2, padx=5, pady=5)

    # Call toggle_min_max_step_visibility initially to set visibility based on default type
    toggle_min_max_step_visibility(variable_type_var.get())

def toggle_min_max_step_visibility(selected_type):
    global min_label, max_label, step_label, min_entry, max_entry, step_entry

    if selected_type in ["int", "float"]:
        min_label.grid()
        min_entry.grid()
        max_label.grid()
        max_entry.grid()

        if selected_type == "float":
            step_label.grid()
            step_entry.grid()
        else:
            step_label.grid_remove()
            step_entry.grid_remove()
    else:
        min_label.grid_remove()
        min_entry.grid_remove()
        max_label.grid_remove()
        max_entry.grid_remove()
        step_label.grid_remove()
        step_entry.grid_remove()

def get_default_min_value(var_type):
    if var_type == "boolean":
        return "False"
    elif var_type == "int":
        return "0"
    elif var_type == "float":
        return "0.0"

def get_default_max_value(var_type):
    if var_type == "boolean":
        return "True"
    elif var_type == "int":
        return "100"
    elif var_type == "float":
        return "100.0"

def get_default_step_value(var_type):
    if var_type == "float":
        return "0.1"
    else:
        return ""



# Main GUI
root = tk.Tk()
root.title("Bifact")
root.geometry(f"{WINDOW_WIDTH}x{WINDOW_HEIGHT}")

# Buttons for importing CSV file and generating randomly
import_button = tk.Button(root, text="Import a CSV File", command=import_csv)
import_button.pack()
generate_button = tk.Button(root, text="Generate Randomly", command=generate_random)
generate_button.pack()

root.mainloop()


In [ ]:
class Cause:
	def __init__(self, id1, id2, X, data = Data()):
		self.id1 = id1
		self.id2 = id2
		self.data = data
		self.x1, self.x2 = data.get_values(id1, id2, X)
	

	def BC(self):
		# le code des BCi
		return True


In [ ]:
class Bifact:
	def __init__(self, f = lambda x:1 ,data= Data(), computeY = False):
		self.data = data
		self.f = f
		if computeY:
			self.y = self.f(data)
			self.computeY = True

	def generate_causes(self, id1, id2):
     #pseudo code
		if self.computeY:
			y1, y2 = self.y[id1], self.y[id2]
		else:
			y1, y2 = self.f(self.data[id1]),self.f(self.data[id2])
		for i in range(1, len(self.data.__domains)):
			for combine in self.data.V():
				cause = Cause(id1, id2,combine)
				if cause.BC():
					return True
		return False

In [80]:
]

IndentationError: unexpected indent (3449944442.py, line 147)